In [27]:

!pip install folium



You are using pip version 8.1.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [43]:
confirmed_csv='corona_confirmed.csv'

confirmed_gitpath= 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'


In [44]:

!curl -o $confirmed_csv $confirmed_gitpath


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 46550  100 46550    0     0  46550      0  0:00:01 --:--:--  0:00:01  104k


In [45]:

import pandas as pd

df=pd.read_csv(confirmed_csv)
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,21,22,22,22,24,24,40,40,74,84
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,51,55,59,64,70,76,89,104,123,146
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,54,60,74,87,90,139,201,230,264,302
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,2,39,39,53,75,88,113,133,164,188
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,1,2,2,3,3,3


In [47]:
df=df.melt(id_vars=["Province/State","Country/Region","Lat","Long"],
           var_name="Date",
           value_name="Cases")
df.head()

,Province/State,Country/Region,Lat,Long,Date,Cases
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [50]:
df['Date']=df['Date'].str.replace(r'(/d+)/(\d+)/(\d+)', r'20\3-\1-\2')
df['Date']=pd.to_datetime(df['Date'])
df.head()

,Province/State,Country/Region,Lat,Long,Date,Cases
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0


In [56]:
df['Province/State']=df['Province/State'].fillna(df['Country/Region'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6905 entries, 0 to 6904
Data columns (total 6 columns):
Province/State    6905 non-null object
Country/Region    6905 non-null object
Lat               6905 non-null float64
Long              6905 non-null float64
Date              6905 non-null datetime64[ns]
Cases             6905 non-null int32
dtypes: datetime64[ns](1), float64(2), int32(1), object(2)
memory usage: 296.8+ KB


In [53]:
df['Cases']=df['Cases'].astype(int)
df=df[df['Cases']>0].reset_index(drop=True)
df.head()

,Province/State,Country/Region,Lat,Long,Date,Cases
0,Anhui,China,31.8257,117.2264,2020-01-22,1
1,Beijing,China,40.1824,116.4142,2020-01-22,14
2,Chongqing,China,30.0572,107.8740,2020-01-22,6
3,Fujian,China,26.0789,117.9874,2020-01-22,1
4,Guangdong,China,23.3417,113.4244,2020-01-22,26


In [58]:
df_alarming_cities=df.sort_values(by='Cases', ascending=False).groupby('Country/Region').head(1).reset_index(drop=True)
df_alarming_cities=df_alarming_cities.head(n=10)
df_alarming_cities

,Province/State,Country/Region,Lat,Long,Date,Cases
0,Italy,Italy,43.0000,12.0000,2020-03-25,74386
1,Hubei,China,30.9756,112.2707,2020-03-24,67801
2,US,US,37.0902,-95.7129,2020-03-25,65778
3,Spain,Spain,40.0000,-4.0000,2020-03-25,49515
4,Germany,Germany,51.0000,9.0000,2020-03-25,37323
5,Iran,Iran,32.0000,53.0000,2020-03-25,27017
6,France,France,46.2276,2.2137,2020-03-25,25233
7,Switzerland,Switzerland,46.8182,8.2275,2020-03-25,10897
8,United Kingdom,United Kingdom,55.3781,-3.4360,2020-03-25,9529
9,"Korea, South","Korea, South",36.0000,128.0000,2020-03-25,9137


In [65]:
import math

total_incidents=df['Cases'].sum()
def geojsons(df):
    features=[]
    for _, row in df.iterrows():
        feature ={
            'type':'Feature',
            'geometry':{
                'type':'Point',
                'coordinates':[row['Long'],row['Lat']]
        },
        'properties':{
            'time':pd.to_datetime(row['Date'],format='%Y-%m-%d').__str__(),
            'style':{'color':''},
            'icon':'circle',
            'iconstyle':{
                'fillColor':'red',
                'fillOpacity':0.8,
                'stroke':'true',
                'radius': math.log(row['Cases'])
                        }
                      }
       }
        features.append(feature)
    return features
start_geojson=geojsons(df)
start_geojson

[{'geometry': {'coordinates': [117.2264, 31.8257], 'type': 'Point'},
  'properties': {'icon': 'circle',
   'iconstyle': {'fillColor': 'red',
    'fillOpacity': 0.8,
    'radius': 0.0,
    'stroke': 'true'},
   'style': {'color': ''},
   'time': '2020-01-22 00:00:00'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [116.4142, 40.1824], 'type': 'Point'},
  'properties': {'icon': 'circle',
   'iconstyle': {'fillColor': 'red',
    'fillOpacity': 0.8,
    'radius': 2.6390573296152584,
    'stroke': 'true'},
   'style': {'color': ''},
   'time': '2020-01-22 00:00:00'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [107.874, 30.0572], 'type': 'Point'},
  'properties': {'icon': 'circle',
   'iconstyle': {'fillColor': 'red',
    'fillOpacity': 0.8,
    'radius': 1.791759469228055,
    'stroke': 'true'},
   'style': {'color': ''},
   'time': '2020-01-22 00:00:00'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [117.9874, 26.0789], 'type': 'Point'},
  'properties': {'icon': 'circ

In [73]:
import folium
from folium.plugins import TimestampedGeoJson

m=folium.Map(
    location=[50,30],
    zoom_start=2,
    tiles='Stamen Toner'
)
for _, row in df_alarming_cities.iterrows():
    
    folium.Marker(
    location=[row['Lat'],row['Long']],
    icon=folium.Icon(color='black',icon='ambulance', prefix='fa'),
    popup=row['Province/State']).add_to(m)
TimestampedGeoJson(
    start_geojson,
    period='P1D',
    duration='PT1M',
    transition_time=2000,
    auto_play=True,
).add_to(m)
m